## D'NOVA YouTube Shorts Scraping
-  Test code: 테스트 코드
- Daily Shorts check: 수집 데이터 loading
#### Code Flow
1. channel shorts url input으로 들어갈시 채널 url 필터링(daily)
2. Shorts 생성 안할시 data_yn n으로 변경
3. 수집한 channel list 수집 후 loaded_db에 적재 
4. 기존 수집했던 쇼츠 재수집 -7d, -14d, -21d

- Sorting 기준 만들어야함 query, view 기준 sorting


### Test Code

In [1]:
import requests
import re
import numpy as np
from bs4 import BeautifulSoup

In [2]:
# Define the payload and headers for the request
# shorts channel1: https://www.youtube.com/watch?v=xEuH_5Ik92U
# shorts channel2: https://www.youtube.com/watch?v=fa_sSaHxTZ4
# yt_shorts page: https://www.youtube.com/@%EC%95%84%EC%9D%B4%EB%8F%8C/shorts
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
payload = {
    'api_key': API_KEY,
    'url': 'https://www.youtube.com/watch?v=fa_sSaHxTZ4'
    ,'render': 'true'
}
# payload = { 'api_key': '3397e8a6a37e89b03b08750a27575df8', 'url': 'https://www.youtube.com/watch?v=xEuH_5Ik92U' }
# r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
# print(r.text)
headers = {
    'x-sapi-render': 'true',
    'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 3}]'
    # , {"type": "click", "selector": {"type": "text","value": "Show transcript"}}
    # ,{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2} 
}
# Make the GET request
r_full = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False) # headers=headers,

# Save the response text to a file
with open('response_channel_test.txt', 'w', encoding='utf-8') as file:
    file.write(r_full.text)

print("Response saved to response_output.txt")


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response saved to response_output.txt


In [3]:
# url 한개만 추출 -> comments (x)
import requests

payload = { 'api_key': '3397e8a6a37e89b03b08750a27575df8', 'url': 'https://www.youtube.com/@%EC%95%84%EC%9D%B4%EB%8F%8C/shorts'}
r_normal = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
# Save the response text to a file
with open('response_channel_test_subscirbers.txt', 'w', encoding='utf-8') as file:
    file.write(r_normal.text)

print("Response saved to response_output.txt")

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response saved to response_output.txt


In [112]:
# url 여러개 진행
import requests
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
# List of URLs to test
urls = [
    'https://www.youtube.com/watch?v=xEuH_5Ik92U',
    'https://www.youtube.com/watch?v=CPsCEfs3m8I',
    'https://www.youtube.com/watch?v=jXUHFbNo5Vg'
]

# API key and headers
headers = {
    'x-sapi-render': 'true',
    # 'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
    'x-sapi-instruction_set': '[{"type":"loop","for":' + str(len(urls))+',"instructions":[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]}]'
}

# Loop through each URL, send request, and save response to a file
for i, url in enumerate(urls):
    payload = {
        'api_key': API_KEY,
        'url': url,
        'render': 'true'
    }
    
    # Send the request
    r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
    
    # Save the response to a file, naming it based on the URL index
    filename = f'response_{i + 1}.txt'
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(r.text)
    
    print(f'Response for {url} saved to {filename}')



c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=xEuH_5Ik92U saved to response_1.txt


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=CPsCEfs3m8I saved to response_2.txt


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=jXUHFbNo5Vg saved to response_3.txt


Start

## Daily & Weekly Shorts Check
- 기본 데이터 loading
- 7687 채널 매일 확인 KST 23:30 진행 -> request 7687 - (no shorts) 기본
- channel_df: 기본 YT 채널 데이터
- loaded_df: 쇼츠 적재 데이터 (추후 DB에 저장 필요)
- basic_shorts_info: 기본적인 Shorts 데이터 추출 함수

In [31]:
# 기존 channel list와 적재 데이터 load
import requests
import re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
# 한국, 미국 데이터 9507개
channel_df = pd.read_excel('us_korea_test.xlsx')
# channel_df = ch_df[81:]
channel_df = channel_df.drop_duplicates(subset=['channel_name', 'yt_fixed_url'], keep='last')
channel_df.to_excel('us_korea_test.xlsx', index=False)
channel_df = channel_df[8000:8010]
# 적재 DB: # -7d, -14d, -21d 만 필터링
loaded_df = pd.read_excel("loaded_db.xlsx")
today = datetime.today().strftime('%Y-%m-%d')
try:
    loaded_df['check_date'] = loaded_df['check_date'].dt.date
    filter_dates = [today - timedelta(days=7), today - timedelta(days=14), today - timedelta(days=21)]
    # Filter the dataframe based on check_date
    loaded_df = loaded_df[loaded_df['check_date'].isin(filter_dates)]
    # channel_df 와 loaded_df 교집합 데이터
    merged_df = pd.merge(loaded_df, channel_df, on='channel_name', how='inner')
except Exception:
    pass

##### 기본 shorts 데이터 수집

In [34]:
# Test 용
# r_full: render 조건 True, request: 10, 쇼츠 데이터
# r_normal: render 조건 False, request: 1, 쇼츠데이터
# r_page: 채널 쇼츠 페이지, reqeust: 1

# scraperAPI result filter with NaN values for missing data
def basic_shorts_info(text):
    # shorts title
    match_title = re.search(r'<title>(.*?) - YouTube</title>', text)
    extracted_title = match_title.group(1) if match_title else np.nan

    # shorts description
    description_pattern = r'attributedDescription":{"content":"(.*?)"'
    # Find the match
    description_match = re.search(description_pattern, text)
    # Extract the content if found
    extracted_description = description_match.group(1) if description_match else None
    
    # shorts total view (date and time)
    date_time_pattern = r'datePublished" content="(\d{4}-\d{2}-\d{2})T(\d{2}):'
    match_date = re.search(date_time_pattern, text)
    extracted_published = f"{match_date.group(1)} {match_date.group(2)}" if match_date else np.nan

    # Thumbnail
    thumbnail_pattern = r'"url":"(https?://[^\s"]+)"'
    thumbnail_match = re.search(thumbnail_pattern, text)
    yt_shorts_thumbnail = thumbnail_match.group(1) if thumbnail_match else np.nan

    # Total view
    view_pattern = r'"viewCountEntityKey":"[^"]+","factoid":\{"factoidRenderer":\{"value":\{"simpleText":"([\d,]+)"'
    view_match = re.search(view_pattern, text)
    extracted_view = view_match.group(1) if view_match else np.nan

    # Total likes
    like_pattern = r'"iconName":"LIKE","title":"([\d.]+)([KM]?)"'
    like_match = re.search(like_pattern, text)
    if like_match:
        extracted_likes = float(like_match.group(1))
        suffix = like_match.group(2)
        if suffix == 'K':
            extracted_likes *= 1000
        elif suffix == 'M':
            extracted_likes *= 1000000
    else:
        extracted_likes = np.nan

    # Total comments    
    comment_num_pattern = r'"text":"Comments"}\]},"contextualInfo":\{"runs":\[\{"text":"([\d.]+)([KM]?)"'
    comment_num_match = re.search(comment_num_pattern, text)
    if comment_num_match:
        extracted_comments_number = float(comment_num_match.group(1))
        suffix = comment_num_match.group(2)
        if suffix == 'K':
            extracted_comments_number *= 1000
        elif suffix == 'M':
            extracted_comments_number *= 1000000
    else:
        extracted_comments_number = np.nan

    # yt shorts comment 결과 및 comment 작성 시간
    comments_time_text_pattern = r'<span[^>]*id="published-time-text"[^>]*>\s*<a[^>]*>\s*(.*?)\s*<\/a>.*?<yt-attributed-string[^>]*>\s*<span[^>]*class="yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap"[^>]*>\s*(.*?)\s*<\/span>'
    
    # Find all matches for time and comments
    comments_time_matches = re.findall(comments_time_text_pattern, text, re.DOTALL)
    
    # Process and clean comments
    extracted_comments_time_matches = []
    if comments_time_matches:
        for time, comment in comments_time_matches:
            # Clean the comment to remove unwanted tags using BeautifulSoup
            soup = BeautifulSoup(comment, 'html.parser')
            cleaned_comment = soup.get_text()
            extracted_comments_time_matches.append((time.strip(), cleaned_comment.strip()))
    else:
        extracted_comments_time_matches = np.nan

    return (extracted_title, extracted_description, extracted_published, yt_shorts_thumbnail, 
            extracted_view, extracted_likes, extracted_comments_number, extracted_comments_time_matches)

In [35]:
# yt_shorts_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r_normal.text)
# print("yt_shorts_title: ", yt_shorts_title)
# print("yt_shorts_description: ", yt_shorts_description)
# print("yt_shorts_published: ", yt_shorts_published)
# print("yt_shorts_thumbnalil: ", yt_shorts_thumbnail)
# print("yt_view: ", yt_shorts_view)
# print("yt_likes: ", yt_shorts_likes)
# print("yt_comments: ", yt_shorts_comments_num)
# print("yt_shorts_comments_time: ", yt_shorts_comments_time)
# print(len(yt_shorts_comments_time))

##### 1. channel shorts url input으로 들어갈시 채널 url 필터링(daily)
- 최신 기준으로 맵핑이 되지 않는 경우 최근 url 1개 수집 
- 기존 말고 신규 shorts url 확인시 수집 list에 추가 -> (아직 테스트 안됨)
- 마지막에 다 확인후 추가된 list 쇼츠 데이터 수집 진행

In [62]:
import pandas as pd
import requests
import re

def convert_subscribers(subscriber_text):
    """Convert subscriber text to a numeric value."""
    if 'K' in subscriber_text:
        return int(float(subscriber_text.replace('K', '').strip()) * 1000)
    elif 'M' in subscriber_text:
        return int(float(subscriber_text.replace('M', '').strip()) * 1000000)
    return int(subscriber_text.strip())

def extract_total_video_count(video_text):
    """Extract total video count as an integer from the input text."""
    number_pattern = r'(\d+) videos'
    match = re.search(number_pattern, video_text)
    return int(match.group(1)) if match else None  # Return the number part if found

def extract_description(description_text):
    """Extract description from the HTML content."""
    lines = description_text.splitlines()
    first_line = lines[0].strip() if len(lines) > 0 else None
    second_line = lines[1].strip() if len(lines) > 1 else None
    return first_line, second_line  # Get the first two lines

def scanning_yt_shorts_page(api_key, channel_db, loaded_db):
    # List to store new shorts results in the format (channel_name, new_shorts_url)
    new_shorts_urls = []
    # List to store channels with no shorts_urls
    no_shorts_list = []
    
    # Initialize a list for the additional DataFrame
    additional_data = []

    # Iterate over each row in channel_db
    for index, row in channel_db.iterrows():
        # Only proceed if data_yn is "y"
        if row['data_yn'] != 'y':
            continue  # Skip the row if data_yn is not "y"
        
        channel_name = row['channel_name']
        yt_fixed_url = row['yt_fixed_url']
        print(channel_name)
        # Filter loaded_db where channel_name matches
        matching_rows = loaded_db[loaded_db['channel_name'] == channel_name]
        
        if not matching_rows.empty:
            # Sort by check_date to get the most recent row
            latest_row = matching_rows.sort_values(by='check_date', ascending=False).iloc[0]
            latest_url = latest_row['shorts_url']
        else:
            # If no match found in loaded_db, set latest_url to None to ensure data collection proceeds
            latest_url = None
        
        # Prepare the payload for ScraperAPI request, using yt_fixed_url
        payload = {
            'api_key': api_key,
            'url': yt_fixed_url
        }
        
        # Send the request to ScraperAPI
        r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
        
        # Define the regex pattern to extract Shorts URLs
        shorts_url_pattern = r'webCommandMetadata":{"url":"(/shorts/\w+)"'
        # Find all matching Shorts URLs
        shorts_urls = re.findall(shorts_url_pattern, r.text)
        shorts_urls = ['https://www.youtube.com' + url.replace('/shorts/', '/watch?v=') for url in shorts_urls]

        print(f"shorts_urls for {channel_name}: ", shorts_urls)

        # Extract subscribers
        subscribers_pattern = r'(\d+K|\d+M|\d+) subscribers'
        subscribers_match = re.search(subscribers_pattern, r.text)
        last_subscribers = convert_subscribers(subscribers_match.group(1)) if subscribers_match else None
        
        # Extract total video count with the required prefix
        total_video_count_pattern = r'{"text":{"content":"(\d+) videos"'
        total_video_count_match = re.search(total_video_count_pattern, r.text)
        last_total_videos = extract_total_video_count(total_video_count_match.group(0)) if total_video_count_match else None
        
        # Extract description
        description_pattern = r'itemprop="description" content="(.*?)"'
        description_match = re.search(description_pattern, r.text, re.DOTALL)
        last_description = extract_description(description_match.group(1)) if description_match else (None, None)

        # Store the data for the DataFrame
        additional_data.append({
            'channel_name': channel_name,
            'yt_fixed_url': yt_fixed_url,
            'subscribers': last_subscribers,
            'total_video_count': last_total_videos,
            'description': last_description[0]  # Assuming we want the first line of the description
        })

        # Proceed based on shorts_urls availability
        if shorts_urls:
            if latest_url and latest_url in shorts_urls:
                # If there's a match, get all URLs before the latest one
                matching_index = shorts_urls.index(latest_url)
                for shorts_url in shorts_urls[:matching_index]:
                    new_shorts_urls.append((channel_name, shorts_url))
            else:
                # If no latest URL or no match in loaded_db, collect the first URL
                new_shorts_urls.append((channel_name, shorts_urls[0]))
        else:
            # If shorts_urls is empty, add the channel_name and yt_fixed_url to the no_shorts_list
            no_shorts_list.append((channel_name, yt_fixed_url))
    
    # Convert the no_shorts_list to a DataFrame
    no_shorts_df = pd.DataFrame(no_shorts_list, columns=['channel_name', 'yt_fixed_url'])
    
    # Create a DataFrame from the additional data
    df = pd.DataFrame(additional_data)

    return new_shorts_urls, no_shorts_df, df  # Return both the list, no_shorts DataFrame, and additional DataFrame


In [63]:
new_shorts_urls = scanning_yt_shorts_page('3397e8a6a37e89b03b08750a27575df8', channel_df, loaded_df)
new_shorts_urls

Everyday Astronaut


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Everyday Astronaut:  ['https://www.youtube.com/watch?v=Hlj_PkYvfnc', 'https://www.youtube.com/watch?v=9yii_nvXaPM', 'https://www.youtube.com/watch?v=RaztA75ge1A', 'https://www.youtube.com/watch?v=UIe8e_LCN64', 'https://www.youtube.com/watch?v=zTDeu8q6JzI', 'https://www.youtube.com/watch?v=F4prIyQ0xvw', 'https://www.youtube.com/watch?v=nRob7JKEZQw', 'https://www.youtube.com/watch?v=WY73exaVpyw', 'https://www.youtube.com/watch?v=zJY7NIzOmC8', 'https://www.youtube.com/watch?v=dDcXytGd29k', 'https://www.youtube.com/watch?v=0mX5H_Gaicg', 'https://www.youtube.com/watch?v=c7AHkuUqlFI', 'https://www.youtube.com/watch?v=352P0sLMnsw']
BizanIQ


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for BizanIQ:  ['https://www.youtube.com/watch?v=NgHEba7SFKY', 'https://www.youtube.com/watch?v=mdDJRxxfLtM', 'https://www.youtube.com/watch?v=ELJ20Ynu9Eg', 'https://www.youtube.com/watch?v=7Z1noV0PxZ8', 'https://www.youtube.com/watch?v=BmAX8Bt5Qos', 'https://www.youtube.com/watch?v=dGfKFUELOMo', 'https://www.youtube.com/watch?v=G6KAzBYPl78', 'https://www.youtube.com/watch?v=q6yicPkQrvk', 'https://www.youtube.com/watch?v=kIlf_S1XB70', 'https://www.youtube.com/watch?v=CjIBGI61QLs', 'https://www.youtube.com/watch?v=i6a2yXEmypY', 'https://www.youtube.com/watch?v=4LOAfaxTHz0', 'https://www.youtube.com/watch?v=3z4IWrnUzDw', 'https://www.youtube.com/watch?v=tWt5E8Q0PKY', 'https://www.youtube.com/watch?v=lt1tuALczts', 'https://www.youtube.com/watch?v=loNwNndL8h4', 'https://www.youtube.com/watch?v=HwOAjiv8I_A', 'https://www.youtube.com/watch?v=5UTQxHgqyfo', 'https://www.youtube.com/watch?v=E075vZGHQPE', 'https://www.youtube.com/watch?v=eN1PVAoxNaQ', 'https://www.youtube.com/watch?v=

...

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Mech Fanatic:  ['https://www.youtube.com/watch?v=X2U25iaQk0g', 'https://www.youtube.com/watch?v=sNRYvfCrD1Y', 'https://www.youtube.com/watch?v=L_W1V2T0DqM', 'https://www.youtube.com/watch?v=p1f4XDqleFw', 'https://www.youtube.com/watch?v=4siOFpmwmQc', 'https://www.youtube.com/watch?v=0_jfdOEbWvU', 'https://www.youtube.com/watch?v=2I6tT7O2D2U', 'https://www.youtube.com/watch?v=B3Sk4BWHHtg', 'https://www.youtube.com/watch?v=iYz0K6IBBg8', 'https://www.youtube.com/watch?v=T6DWk7t19a8', 'https://www.youtube.com/watch?v=npD0BJ3e_C8', 'https://www.youtube.com/watch?v=AN5WpfzMfE0', 'https://www.youtube.com/watch?v=7T2lofqD1dc', 'https://www.youtube.com/watch?v=Sza0wsTfovc', 'https://www.youtube.com/watch?v=kZZM1jZ9g2o', 'https://www.youtube.com/watch?v=zKiO3HQUbPc', 'https://www.youtube.com/watch?v=RLDNziaKMeg', 'https://www.youtube.com/watch?v=bTZfR7rtRY4', 'https://www.youtube.com/watch?v=2R7PFS9EGSo', 'https://www.youtube.com/watch?v=a_SxEGnYHI4', 'https://www.youtube.com/wat

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Action Lab Shorts:  ['https://www.youtube.com/watch?v=WFO4G4KFAro', 'https://www.youtube.com/watch?v=zBPdhPgp86M', 'https://www.youtube.com/watch?v=3cF2nmnYkak', 'https://www.youtube.com/watch?v=Hdnkd1VuEiY', 'https://www.youtube.com/watch?v=PgRPJcaLlxg', 'https://www.youtube.com/watch?v=VNvhlzXFRlw', 'https://www.youtube.com/watch?v=ovbrJ1YlUeg', 'https://www.youtube.com/watch?v=P4_Xj_muLkQ', 'https://www.youtube.com/watch?v=tXanxBSroLI', 'https://www.youtube.com/watch?v=CL_74thRzuU', 'https://www.youtube.com/watch?v=Gj79I9Z4EQY', 'https://www.youtube.com/watch?v=pjnpCrVnGu4', 'https://www.youtube.com/watch?v=FkwVZm8ufcs', 'https://www.youtube.com/watch?v=_35QwNl3Ldg', 'https://www.youtube.com/watch?v=ODZ9oymVvMU', 'https://www.youtube.com/watch?v=IOilMSsbK2Q', 'https://www.youtube.com/watch?v=CTFEKsmbo7s', 'https://www.youtube.com/watch?v=8OAJwsfD3XE', 'https://www.youtube.com/watch?v=7G1hTveKI1s', 'https://www.youtube.com/watch?v=SRO5JiwFYd4', 'https://www.youtube.co

([('Everyday Astronaut', 'https://www.youtube.com/watch?v=Hlj_PkYvfnc'),
  ('BizanIQ', 'https://www.youtube.com/watch?v=NgHEba7SFKY'),
  ('Mr. Leaf Area', 'https://www.youtube.com/watch?v=X5YlCbKDPaQ'),
  ('AuthenTech - Ben Schmanke', 'https://www.youtube.com/watch?v=P4eFD4_1NHo'),
  ('Google Workspace', 'https://www.youtube.com/watch?v=Yq0DNuD_6zQ'),
  ('Earth Untamed ', 'https://www.youtube.com/watch?v=GvxU5MM9f_c'),
  ('Spark Cycleworks', 'https://www.youtube.com/watch?v=awCLlLMAcB8'),
  ('Explore Infinity', 'https://www.youtube.com/watch?v=auPl6hPvEjM'),
  ('Mech Fanatic', 'https://www.youtube.com/watch?v=X2U25iaQk0g'),
  ('Action Lab Shorts', 'https://www.youtube.com/watch?v=WFO4G4KFAro')],
 Empty DataFrame
 Columns: [channel_name, yt_fixed_url]
 Index: [],
                 channel_name  \
 0         Everyday Astronaut   
 1                    BizanIQ   
 2              Mr. Leaf Area   
 3  AuthenTech - Ben Schmanke   
 4           Google Workspace   
 5             Earth Untamed 

#### YT 채널홈 shorts페이지에서 추가 데이터 수집
- 채널 설명
- 채널 구독자
- 채널 총 비디오

In [89]:
def update_channel_df_with_new_data(ch_df, new_data):
    """
    Update the channel DataFrame with new data based on matching channel names.
    
    Parameters:
    ch_df (pd.DataFrame): The original channel DataFrame to be updated.
    new_data (pd.DataFrame): The new data DataFrame containing updated information.
    
    Returns:
    pd.DataFrame: The updated channel DataFrame.
    """
    for index, new_row in new_data.iterrows():
        channel_name = new_row['channel_name']

        # Find matching channel in channel_df
        matching_row = ch_df[ch_df['channel_name'] == channel_name]

        if not matching_row.empty:
            # Update values if they're NaN or if the new value is not NaN
            ch_df.loc[ch_df['channel_name'] == channel_name, 'subscribers'] = (
                new_row['subscribers'] if pd.notna(new_row['subscribers']) else ch_df.loc[ch_df['channel_name'] == channel_name, 'subscribers']
            )
            
            ch_df.loc[ch_df['channel_name'] == channel_name, 'total_video_count'] = (
                new_row['total_video_count'] if pd.notna(new_row['total_video_count']) else ch_df.loc[ch_df['channel_name'] == channel_name, 'total_video_count']
            )
    
    return ch_df

# Usage example:
# updated_channel_df = update_channel_df_with_new_data(channel_df, new_shorts_df)
# print(updated_channel_df) 

updated_channel_df = update_channel_df_with_new_data(channel_df, new_shorts_urls[2])
updated_channel_df


,youtuberme_url,category,country,channel_name,subscribers,total_video_views,total_video_count,started,yt_url,yt_fixed_url,data_yn
0,NaN,science-technology,United States,Everyday Astronaut,73000000.0,NaN,457.0,NaN,NaN,https://www.youtube.com/channel/UC6uKrU_WqJ1R2...,y
1,NaN,science-technology,United States,BizanIQ,294000.0,NaN,49.0,NaN,NaN,https://www.youtube.com/channel/UCteEM4YIjpfes...,y
2,NaN,science-technology,United States,Mr. Leaf Area,486000.0,NaN,184.0,NaN,NaN,https://www.youtube.com/channel/UCCaQoZECMg3eL...,y
3,NaN,science-technology,United States,AuthenTech - Ben Schmanke,808000.0,NaN,968.0,NaN,NaN,https://www.youtube.com/channel/UCgyvzxg11MtND...,y
4,NaN,science-technology,United States,Google Workspace,872000.0,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCBmwzQnSoj9b6...,y
5,NaN,science-technology,United States,Earth Untamed,7000.0,NaN,41.0,NaN,NaN,https://www.youtube.com/channel/UCWTX75gsSsBuq...,y
6,NaN,science-technology,United States,Spark Cycleworks,59000.0,NaN,218.0,NaN,NaN,https://www.youtube.com/channel/UCSWGEQ8n_6xMl...,y
7,NaN,science-technology,United States,Explore Infinity,17000.0,NaN,120.0,NaN,NaN,https://www.youtube.com/channel/UCGmqyyyEES62d...,y
8,NaN,science-technology,United States,Mech Fanatic,4000.0,NaN,101.0,NaN,NaN,https://www.youtube.com/channel/UCCDS_HHq00aCH...,y
9,NaN,science-technology,United States,Action Lab Shorts,2000000.0,NaN,553.0,NaN,NaN,https://www.youtube.com/channel/UCA19mAJURyYHb...,y


#### 2. YouTube 채널 및 shorts를 업로드 하지 않는 경우
- data_yn에서 y를 n으로 변경 (y가 default)
- 필터링 중 n이였던 건이 y가 되는 경우도 체크

In [ ]:
def update_channel_df_with_new_shorts(df, new_shorts_urls_df, update_df):
    # Iterate over each row in new_shorts_urls_df to match with channel_df
    for index, row in new_shorts_urls_df.iterrows():
        target_channel_name = row['channel_name']
        
        # Update 'data_yn' in channel_df where 'channel_name' matches
        df.loc[df['channel_name'] == target_channel_name, 'data_yn'] = 'n'

    # Return the updated channel_df with all rows
    return df

updated_channel_df = update_channel_df_with_new_shorts(channel_df, new_shorts_urls[1])

# Display the updated DataFrame
updated_channel_df

,youtuberme_url,category,country,channel_name,subscribers,total_video_views,total_video_count,started,yt_url,yt_fixed_url,data_yn
40,https://us.youtubers.me//satori-graphics/youtu...,education,United States,Satori Graphics,1300000,77806503,1090,2014,https://us.youtubers.me//satori-graphics/youtube,https://www.youtube.com/channel/UCoeJKtPJLoIBq...,Y
41,https://us.youtubers.me//education-world/youtu...,education,United States,Education World,1140000,77309884,355,2008,https://us.youtubers.me//education-world/youtube,https://www.youtube.com/channel/UCkgElsjGDR4Pw...,Y
42,https://us.youtubers.me//chris-03bedb0f-1b0a-4...,education,United States,口语老炮儿马思瑞 Laoma Chris,820000,76008232,318,2017,https://us.youtubers.me//chris-03bedb0f-1b0a-4...,https://www.youtube.com/channel/UC8FnQA_ZSeHwx...,Y
43,https://us.youtubers.me//shane-hummus/youtuber...,education,United States,Shane Hummus,745000,75766163,860,2017,https://us.youtubers.me//shane-hummus/youtube,https://www.youtube.com/channel/UCLKZ20yD2tNMB...,Y
44,https://us.youtubers.me//drbeen-medical-lectur...,education,United States,Drbeen Medical Lectures,644000,75581366,1556,2010,https://us.youtubers.me//drbeen-medical-lectur...,https://www.youtube.com/channel/UCrtd2wePvAl6R...,Y
45,https://us.youtubers.me//tony-robbins/youtuber...,education,United States,Tony Robbins,1920000,75150219,615,2010,https://us.youtubers.me//tony-robbins/youtube,https://www.youtube.com/channel/UCJLMboBYME_CL...,Y
46,https://us.youtubers.me//simple-nursing/youtub...,education,United States,Simple Nursing,1190000,74200294,566,2012,https://us.youtubers.me//simple-nursing/youtube,https://www.youtube.com/channel/UCUxQWmWk1_Hk9...,Y
47,https://us.youtubers.me//higher-self/youtuber-...,education,United States,Higher Self,594000,74022822,951,2007,https://us.youtubers.me//higher-self/youtube,https://www.youtube.com/channel/UCBz4vE0S8E1Os...,n
48,https://us.youtubers.me//nate-o-brien/youtuber...,education,United States,Nate O'Brien,1310000,73794475,204,2016,https://us.youtubers.me//nate-o-brien/youtube,https://www.youtube.com/channel/UCO3tlaeZ6Z0ZN...,n
49,https://us.youtubers.me//ballistic-high-speed/...,education,United States,Ballistic High-Speed,586000,73350166,60,2021,https://us.youtubers.me//ballistic-high-speed/...,https://www.youtube.com/channel/UCxdMiqKX7PQuw...,Y


In [ ]:
# test로 진행한 일부 변경
channel_df.update(updated_channel_df)
channel_df.to_excel("us_korea_test.xlsx", index=False)

##### 3. 수집한 channel list 수집 후 loaded_db에 적재
- new_shorts_urls 기준으로 rendered=True로 수집
- 수집 데이터 loaded_db에 적재 후 저장

In [ ]:
# url 여러개 진행
def add_new_shorts_data(api_key, urls):
    data_list = []
    check_date = today
    # Separate channel names and shorts URLs
    channel_names = [item[0] for item in urls]
    shorts_urls = [item[1] for item in urls]
    
    for channel_name, url in zip(channel_names, shorts_urls):
        print(channel_name, url)
        shorts_url = url
        # Prepare payload and headers for the request
        payload = {
            'api_key': api_key,
            'url': url
            # ,'render': 'true'
        }
        # Make the GET request
        r = requests.get('https://api.scraperapi.com/', params=payload,  verify=False)
        # Extract the necessary data
        shorts_title, shorts_description, shorts_published, shorts_thumbnail, shorts_view, shorts_likes, shorts_comments_num, shorts_comments_time = basic_shorts_info(r.text)
        
        # Print the extracted information for debugging
        # print("title: ", channel_name)
        # print("shorts_title: ", shorts_title)
        # print("shorts_description: ", shorts_description)
        # print("shorts_published: ", shorts_published)
        # print("shorts_thumbnalil: ", shorts_thumbnail)
        # print("view: ", shorts_view)
        # print("likes: ", shorts_likes)
        # print("comments: ", shorts_comments_num)
        # print("shorts_comments_time: ", shorts_comments_time)

        # Append as a tuple to the data_list
        data_list.append((
            channel_name,
            shorts_url, 
            shorts_title,  
            shorts_description, 
            shorts_thumbnail, 
            shorts_view, 
            shorts_likes, 
            shorts_comments_num, 
            shorts_comments_time,
            shorts_published, 
            check_date

        ))
    
    return data_list

data_list = add_new_shorts_data(API_KEY, new_shorts_urls[0])
print(data_list)



Financian https://www.youtube.com/watch?v=y0I5XJ19L2s


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


title:  Financian
shorts_title:  Why Men Won&#39;t Order Cocktail Without Any Images
shorts_description:  Rory Sutherland tells why people won't order cocktails in the bar without images in the menu.\n\nDon't forget to follow us on Instagram - @financian_\n  / financian_  \n\nDon't forget to follow us on TikTok - @financian_\n  / financian_  \n\n#short #shorts
shorts_published:  2024-10-28 14
shorts_thumbnalil:  https://rr2---sn-pouxga5o-vu2s.googlevideo.com/videoplayback?expire=1730211691\u0026ei=C5sgZ_X2LZmFi9oPyrblgAM\u0026ip=45.93.225.12\u0026id=o-AD6FhaRrSFDjcKJigOCcUo4eUpf6-ml5CXmoy617Fyso\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1730190091%2C\u0026mh=gj\u0026mm=31%2C29\u0026mn=sn-pouxga5o-vu2s%2Csn-5hne6nsz\u0026ms=au%2Crdu\u0026mv=u\u0026mvi=2\u0026pl=24\u0026rms=au%2Cau\u0026bui=AQn3pFSlSNbS_hbh4jPBfhQtHyPYvyagnrEoS__2VMDNVPZNV7Cw5L5eA6KeVtkrBz8l3gMj4D6x0Fez\u0026spc=qtApAWmu4tvRje0hBVLJJOhPEt4m385fZ9mqf_cp3ZLpM_K5xiXJgBOla-qmFFs\

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


title:  Financian
shorts_title:  
shorts_description:  Rory Sutherland tells how using a simple marketing trick they made their letters important to clients and why banks are building their buildings so massive and beautiful.\n\nRory Sutherland brilliantly demonstrates the impact of perceived value in marketing by recounting how they transformed a simple communication into something that commanded attention. Typically, companies send out emails or standard mail to convey information, but these can easily be ignored or dismissed. Instead of following this predictable route, Sutherland and his team employed a clever strategy: they started sending otherwise generic letters through FedEx and UPS. This added a tangible layer of significance to each message, conveying to the recipient that the content must be important if the company invested in a premium delivery service.\n\nThis approach leverages the psychology of perceived effort and cost. When someone receives a letter through FedEx or 

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


title:  Umar Ashraf
shorts_title:  Stop treating trading like a get rich quick scheme
shorts_description:  I use TradeZella to track and journal my trades.\nhttps://www.tradezella.com/?via=umar\n\nFollow my socials: \n\nInstagram -   / umarashraf  \nTwitter -   / umarashraf  \n\n\n\n#daytrading #daytrader #trading
shorts_published:  2024-10-28 13
shorts_thumbnalil:  https://rr1---sn-1gi7znek.googlevideo.com/videoplayback?expire=1730211712\u0026ei=IJsgZ6LaMdyCi9oPhaXOoQk\u0026ip=168.199.227.27\u0026id=o-AMvXGZz9BxQ3imjl8XbFreXPrNFkAmUhYpqqG2ZK05ho\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1730190112%2C\u0026mh=ry\u0026mm=31%2C29\u0026mn=sn-1gi7znek%2Csn-1gieen7e\u0026ms=au%2Crdu\u0026mv=m\u0026mvi=1\u0026pl=24\u0026rms=au%2Cau\u0026initcwndbps=15801250\u0026bui=AQn3pFQVHNBXZo53IkZ3SAJOzxItqOUKDdsprDJwup_5z7ZrJ5QKt6hWpukUI4EubLp6XMuiHR_lJQCX\u0026spc=qtApATys88DmDNs60i-MvUvSd5ERsVrFWYUXbwVhl-xdg0jlPQH6J8adQMLnvc0\u0026vprv=1\u0026svpuc=1\u002

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


title:  Jaalhaq
shorts_title:  বড় বড় বিপদ থেকে মুক্তির ওজিফা  #smnizamuddin #dua #wazifa #islamicvideo
shorts_description:  বড় বড় বিপদ থেকে মুক্তির ওজিফা  #smnizamuddin #dua #wazifa #islamicvideo \n\nআমাদের সকল চ্যানেল ও পেজ লিংক\n   / @allbangladua  \n   / @allahwalamedia  \n   / @jaalhaq  \n   / @labibvoice  \n   / @muslimgirlmedia  \n\n  / jaalhaqu  \n  / allbangladua  \n  / karamatulauliya  \n  / jumakhutbabd   \n\nhttps://myjaalhaq.blogspot.com\nhttps://allbangladua.blogspot.com\nhttps://12monthkhutba.blogspot.com\n\n  / jaalhaq.bd  \n  / allbangladua  \n\n#jaalhaq #islamic #ইসলামিক #smnizamuddin
shorts_published:  2024-10-28 22
shorts_thumbnalil:  https://rr2---sn-5hne6nsd.googlevideo.com/videoplayback?expire=1730211748\u0026ei=RJsgZ5etDOuAi9oPgs2k8A4\u0026ip=85.237.222.102\u0026id=o-ANItg6E1fYrfSc5iGw-n3LbLIdPaoFhi-ueAUbe24YXn\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1730190148%2C\u0026mh=4q\u0026mm=31%2C26\u0026mn=sn-5hne6nsd%2Csn-

In [ ]:
# 추가된 데이터 dataframe으로 loaed_db에 추가
def add_new_data_to_loaded_db(data_list, loaded_db):
    # Define the columns for the DataFrame
    columns = [
        'channel_name', 'shorts_url', 'shorts_title', 'shorts_description', 
        'shorts_thumbnail', 'shorts_view', 'shorts_likes', 
        'shorts_comments_num', 'shorts_comments_time', 
        'shorts_published_date', 'check_date'
    ]
    
    # Map the data_list to the correct columns and create a new DataFrame
    new_data_df = pd.DataFrame(data_list, columns=columns)
    
    # Append the new data to the loaded_db DataFrame
    updated_loaded_db = pd.concat([loaded_db, new_data_df], ignore_index=True)
    
    return updated_loaded_db

loaded_df = add_new_data_to_loaded_db(data_list, loaded_df)
loaded_df

,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date
0,Robert Breaker,https://www.youtube.com/watch?v=qc0iqmlAR38,NaN,NaN,https://support.google.com/youtube/answer/3037...,"6,422",264.0,16.0,NaN,NaN,2024-10-28
1,Nick Jones,https://www.youtube.com/watch?v=fa_sSaHxTZ4,NaN,NaN,https://support.google.com/youtube/answer/3037...,"91,113",17000.0,553.0,NaN,NaN,2024-10-28
2,Steve Ram,https://www.youtube.com/watch?v=SqU7aeAPkrY,Will You Comply? New Coming Mandates in 2023,FULL VIDEO • TRUMP BLASTS Bringing Back COV...,https://rr4---sn-a5mlrnlz.googlevideo.com/vide...,"56,703",3200.0,504.0,NaN,2023-09-04 15,2024-10-28
3,So Sweet ASMR,https://www.youtube.com/watch?v=k27UsRRjC5g,ASMR Marble run #marblerun #asmr,ASMR Marble run #marblerun #asmr,https://rr1---sn-vgqsknez.googlevideo.com/vide...,"12,582",236.0,3.0,NaN,2024-07-03 08,2024-10-28
4,English with Jennifer,https://www.youtube.com/watch?v=7L3CTSU7MqU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-28
...,...,...,...,...,...,...,...,...,...,...,...
437,Tic Tac Toy Family,https://www.youtube.com/watch?v=ozTkKekTrRA,LEGO Fortnite with the Family !! @LEGOFortnite...,#EpicPartner #MayThe4th #LEGOStarWars #LEGOFor...,https://rr1---sn-pouxga5o-vu2l.googlevideo.com...,"24,837",868.0,31.0,NaN,2024-05-10 04,2024-10-28
438,Financian,https://www.youtube.com/watch?v=y0I5XJ19L2s,Why Men Won&#39;t Order Cocktail Without Any I...,Rory Sutherland tells why people won't order c...,https://rr2---sn-pouxga5o-vu2s.googlevideo.com...,"96,985",4000.0,103.0,NaN,2024-10-28 14,2024-10-29
439,Financian,https://www.youtube.com/watch?v=sOIshfwqEyM,,Rory Sutherland tells how using a simple marke...,https://support.google.com/youtube/answer/3037...,"70,796",2100.0,11.0,NaN,NaN,2024-10-29
440,Umar Ashraf,https://www.youtube.com/watch?v=TmPNbpB8LFI,Stop treating trading like a get rich quick sc...,I use TradeZella to track and journal my trade...,https://rr1---sn-1gi7znek.googlevideo.com/vide...,"5,452",689.0,6.0,NaN,2024-10-28 13,2024-10-29


##### loaded_df 저장

In [ ]:
# 중복 제거
loaded_df = loaded_df.drop_duplicates()
loaded_df.to_excel("loaded_db.xlsx", index=False)

##### 4. 기존 수집했던 쇼츠 재수집 today, -7d, -14d, -21d
- today 데이터 수집 완료후 진행
- loaded_db에서 날짜 필터링해서 데이터 수집 진행

In [ ]:
from datetime import datetime, timedelta

def collecting_past_shorts(api_key, loaded_df):
    print(loaded_df)
    
    # Convert 'today' to a datetime object from the 'yyyy-mm-dd' format
    today_date = datetime.strptime(today, "%Y-%m-%d")
    
    # Iterate over each row in loaded_df, checking check_date and shorts_url
    for idx, (check_dt, shorts_url) in enumerate(zip(loaded_df['check_date'], loaded_df['shorts_url'])):
        # Convert check_dt to datetime format for comparison
        check_dt_date = datetime.strptime(check_dt, "%Y-%m-%d") if isinstance(check_dt, str) else check_dt
        
        # Check if check_dt matches any of the past intervals (7, 14, or 21 days ago)
        if check_dt_date in [today_date - timedelta(days=days) for days in [7, 14, 21]]:
            payload = {
                'api_key': api_key,
                'url': shorts_url
            }
            
            # Send the request
            r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)

            # Extract relevant information from the response
            yt_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r.text)
            
            # Prepare new row with updated values (keeping the same channel name and other values from loaded_df)
            new_row = {
                'channel_nm': loaded_df.loc[idx, 'channel_nm'], 
                'shorts_url': shorts_url, 
                'shorts_title': yt_title,
                'shorts_description': yt_shorts_description,
                'shorts_thumbnail': yt_shorts_thumbnail,
                'shorts_view': yt_shorts_view,
                'shorts_likes': yt_shorts_likes,
                'shorts_comments': yt_shorts_comments_num,
                'shorts_comments_time': yt_shorts_comments_time,
                'shorts_published_date': yt_shorts_published,
                'check_date': today  # Use today's date for the new check_date
            }
            
            # Append the new row to the DataFrame
            loaded_df = loaded_df.append(new_row, ignore_index=True)
        else:
            print("no data")
            pass

    return loaded_df

# Call the function
today_past_shorts = collecting_past_shorts(API_KEY, loaded_df)

              channel_name                                   shorts_url  \
0           Robert Breaker  https://www.youtube.com/watch?v=qc0iqmlAR38   
1               Nick Jones  https://www.youtube.com/watch?v=fa_sSaHxTZ4   
2                Steve Ram  https://www.youtube.com/watch?v=SqU7aeAPkrY   
3            So Sweet ASMR  https://www.youtube.com/watch?v=k27UsRRjC5g   
4    English with Jennifer  https://www.youtube.com/watch?v=7L3CTSU7MqU   
..                     ...                                          ...   
437     Tic Tac Toy Family  https://www.youtube.com/watch?v=ozTkKekTrRA   
438              Financian  https://www.youtube.com/watch?v=y0I5XJ19L2s   
439              Financian  https://www.youtube.com/watch?v=sOIshfwqEyM   
440            Umar Ashraf  https://www.youtube.com/watch?v=TmPNbpB8LFI   
441                Jaalhaq  https://www.youtube.com/watch?v=8o2alUPOm5A   

                                          shorts_title  \
0                                        

In [ ]:
loaded_df

,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date
0,Robert Breaker,https://www.youtube.com/watch?v=qc0iqmlAR38,NaN,NaN,https://support.google.com/youtube/answer/3037...,"6,422",264.0,16.0,NaN,NaN,2024-10-28
1,Nick Jones,https://www.youtube.com/watch?v=fa_sSaHxTZ4,NaN,NaN,https://support.google.com/youtube/answer/3037...,"91,113",17000.0,553.0,NaN,NaN,2024-10-28
2,Steve Ram,https://www.youtube.com/watch?v=SqU7aeAPkrY,Will You Comply? New Coming Mandates in 2023,FULL VIDEO • TRUMP BLASTS Bringing Back COV...,https://rr4---sn-a5mlrnlz.googlevideo.com/vide...,"56,703",3200.0,504.0,NaN,2023-09-04 15,2024-10-28
3,So Sweet ASMR,https://www.youtube.com/watch?v=k27UsRRjC5g,ASMR Marble run #marblerun #asmr,ASMR Marble run #marblerun #asmr,https://rr1---sn-vgqsknez.googlevideo.com/vide...,"12,582",236.0,3.0,NaN,2024-07-03 08,2024-10-28
4,English with Jennifer,https://www.youtube.com/watch?v=7L3CTSU7MqU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-28
...,...,...,...,...,...,...,...,...,...,...,...
437,Tic Tac Toy Family,https://www.youtube.com/watch?v=ozTkKekTrRA,LEGO Fortnite with the Family !! @LEGOFortnite...,#EpicPartner #MayThe4th #LEGOStarWars #LEGOFor...,https://rr1---sn-pouxga5o-vu2l.googlevideo.com...,"24,837",868.0,31.0,NaN,2024-05-10 04,2024-10-28
438,Financian,https://www.youtube.com/watch?v=y0I5XJ19L2s,Why Men Won&#39;t Order Cocktail Without Any I...,Rory Sutherland tells why people won't order c...,https://rr2---sn-pouxga5o-vu2s.googlevideo.com...,"96,985",4000.0,103.0,NaN,2024-10-28 14,2024-10-29
439,Financian,https://www.youtube.com/watch?v=sOIshfwqEyM,,Rory Sutherland tells how using a simple marke...,https://support.google.com/youtube/answer/3037...,"70,796",2100.0,11.0,NaN,NaN,2024-10-29
440,Umar Ashraf,https://www.youtube.com/watch?v=TmPNbpB8LFI,Stop treating trading like a get rich quick sc...,I use TradeZella to track and journal my trade...,https://rr1---sn-1gi7znek.googlevideo.com/vide...,"5,452",689.0,6.0,NaN,2024-10-28 13,2024-10-29


In [ ]:
# 중복 제거
loaded_df = today_past_shorts.drop_duplicates()
loaded_df.to_excel("loaded_db.xlsx", index=False)


#### 5. 고객에게 전달할 Top shorts 데이터 필터링
- 고객 전용 Query 작성
- view 기준으로 ranking

In [6]:
loaded_df

,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date
